In [1]:
print("ok")

ok


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [4]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm_google = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

In [11]:
from langchain_groq import ChatGroq
llm_gorq=ChatGroq(model="mixtral-8x7b")
llm_gorq

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DC77F9C9A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DC77F9CEB0>, model_name='mixtral-8x7b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm_google.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

I am doing well, thank you for asking! How are you today?
Okay, let's dive into the world of Artificial Intelligence (AI)!  It's a vast and rapidly evolving field, so I'll try to give you a comprehensive overview.

**What is Artificial Intelligence?**

At its core, Artificial Intelligence is about creating machines that can perform tasks that typically require human intelligence.  This includes things like:

*   **Learning:** Acquiring information and rules for using the information.
*   **Reasoning:** Using rules to reach conclusions (either definite or probabilistic).
*   **Problem-solving:** Finding solutions to complex issues.
*   **Perception:** Understanding and interpreting sensory input (like images, sound, or text).
*   **Language understanding:** Comprehending and generating human language.

**Key Concepts and Approaches:**

*   **Machine Learning (ML):**  A subset of AI that focuses on enabling systems to learn from data without being explicitly programmed.  Instead of writi

In [5]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [6]:
store={}

In [7]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [9]:
config = {"configurable": {"session_id": "firstchat"}}

In [10]:
model_with_memory=RunnableWithMessageHistory(llm_google,get_session_history)

In [11]:
model_with_memory.invoke(("Hi! I'm Aniruddha Mohanty"),config=config).content

"Hi Aniruddha! It's nice to meet you. How can I help you today?"

In [12]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Aniruddha Mohanty.'

In [13]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Aniruddha Mohanty", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Aniruddha! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--71633516-aefc-419f-ba20-d460d4519967-0', usage_metadata={'input_tokens': 11, 'output_tokens': 22, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Aniruddha Mohanty.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--b632d350-ec64-447c-99f2-a7479521101f-0', usage_metadata={'input_tokens': 39, 'output_tokens': 11, 'total_tokens': 50, 'input_token_details': 

In [20]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [21]:
loader = DirectoryLoader('/data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerUserPerProjectPerModel-FreeTier"
}
viola